Nos conectamos al servidor de jupiter del namenode. Para ello, hay que:
- Activar el entorno virtual con `source venv/bin/activate`
- Conectarse al servidor con `jupyter notebook --ip=0.0.0.0 --port=8888 --no-browser --NotebookApp.token=''`

Una vez tenemos el servidor ejecutando, le damos a `select kernel` en VScode y seleccionamos `existing jupyter server`. En la ventana que aparece, ponemos `http://127.0.0.1:8888/tree` y ya podemos trabajar con el notebook.

Confirmamos que estamos en el directorio correcto `(/home/luser)`

In [ ]:
!pwd

/home/luser


Creamos la varaiable de entorno `CLASSPATH`

In [38]:
import subprocess
import os

# Establecer la variable de entorno
classpath = subprocess.check_output('hdfs classpath --glob', shell=True).decode().strip()
os.environ['CLASSPATH'] = classpath

# Leer la variable de entorno
print(os.environ['CLASSPATH'])

/opt/bd/hadoop/etc/hadoop:/opt/bd/hadoop/share/hadoop/common/lib/netty-resolver-dns-native-macos-4.1.89.Final-osx-aarch_64.jar:/opt/bd/hadoop/share/hadoop/common/lib/netty-buffer-4.1.89.Final.jar:/opt/bd/hadoop/share/hadoop/common/lib/netty-codec-haproxy-4.1.89.Final.jar:/opt/bd/hadoop/share/hadoop/common/lib/netty-codec-memcache-4.1.89.Final.jar:/opt/bd/hadoop/share/hadoop/common/lib/jackson-mapper-asl-1.9.13.jar:/opt/bd/hadoop/share/hadoop/common/lib/commons-math3-3.1.1.jar:/opt/bd/hadoop/share/hadoop/common/lib/jetty-servlet-9.4.51.v20230217.jar:/opt/bd/hadoop/share/hadoop/common/lib/guava-27.0-jre.jar:/opt/bd/hadoop/share/hadoop/common/lib/netty-handler-ssl-ocsp-4.1.89.Final.jar:/opt/bd/hadoop/share/hadoop/common/lib/kerb-crypto-1.0.1.jar:/opt/bd/hadoop/share/hadoop/common/lib/commons-collections-3.2.2.jar:/opt/bd/hadoop/share/hadoop/common/lib/commons-cli-1.2.jar:/opt/bd/hadoop/share/hadoop/common/lib/jaxb-impl-2.2.3-1.jar:/opt/bd/hadoop/share/hadoop/common/lib/jackson-core-asl-1.

# 1. Ejecución de código filesystem_cat

Creamos un fichero de texto de prueba que contenga la siguiente información:
```plaintext
Esto es una prueba de un archivo de texto.

Esto es una prueba de una linea de un archivo de texto.

FIN.
```

In [60]:
# Creamos el contenido del archivo
content = '''
Esto es una prueba de un archivo de texto.

Esto es una prueba de una linea de un archivo de texto.

FIN.
'''
print(content)

# Escribimos el contenido con python
with open('test_2.txt', 'w') as f:
	f.write(content)

# Enseñamos el contenido del archivo con bash
!cat test_2.txt


Esto es una prueba de un archivo de texto.

Esto es una prueba de una linea de un archivo de texto.

FIN.


Esto es una prueba de un archivo de texto.

Esto es una prueba de una linea de un archivo de texto.

FIN.


Movemos el fichero de prueba test.txt a hdfs

In [62]:
%%bash

# Copiamos un fichero local a HDFS
hdfs dfs -put -f test.txt hdfs://namenode:9000/user/luser

# Comprobamos que el fichero se ha copiado correctamente
hdfs dfs -ls hdfs://namenode:9000/user/luser

Found 5 items
drwxr-xr-x   - luser supergroup          0 2024-12-13 18:13 hdfs://namenode:9000/user/luser/.sparkStaging
drwxr-xr-x   - luser supergroup          0 2024-12-04 18:01 hdfs://namenode:9000/user/luser/libros
drwxr-xr-x   - luser supergroup          0 2024-12-13 16:27 hdfs://namenode:9000/user/luser/patentes
drwxr-xr-x   - luser supergroup          0 2024-12-13 16:29 hdfs://namenode:9000/user/luser/patentes-mini
-rw-r--r--   3 luser supergroup        106 2025-01-01 16:46 hdfs://namenode:9000/user/luser/test.txt


Comprobamos que el script filesystem_cat.py funciona correctamente.

In [71]:
!python3 filesystem_cat.py hdfs://namenode:9000/user/luser/test.txt

2025-01-01 16:53:58,492 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Esto es una prueba de un archivo de texto.

Esto es una prueba de una linea de un archivo de texto.

FIN.


# 2. Acceso y gestión de HDFS

Arreglamos el código de copy_half_file.py para que funcione correctamente. El contenido es:
```python
#! /usr/bin/env python3
from pyarrow.fs import FileSystem, FileInfo
import sys
import shutil

def copy_half_file(uri1: str, uri2: str) -> None:
    fs1, path = FileSystem.from_uri(uri1)
    fs2, path2 = FileSystem.from_uri(uri2)

    f_info: FileInfo = fs1.get_file_info(path)  # Obtener información del archivo en fs1

    with fs1.open_input_file(path) as instream, \
         fs2.open_output_stream(path2) as outstream:
        # Mover el puntero de instream a la mitad del archivo
        instream.seek(f_info.size // 2)
        # Copiar el resto del archivo en outstream
        shutil.copyfileobj(instream, outstream)

if __name__ == "__main__":
    # Si no se proporcionan dos argumentos, se muestra un mensaje de error
    if len(sys.argv) != 3:
        print("Uso: {} <uri1> <uri2>".format(sys.argv[0]))
        sys.exit(1)

    copy_half_file(sys.argv[1], sys.argv[2])
```

Creamos un arhcivo .py que contenga el código anterior y lo movemos a hdfs.

In [70]:

# Creamos el archivo .py con el contenido anterior en un directorio con permisos de escritura
with open("/home/luser/copy_half_file.py", "w") as f:
    f.write("""
#! /usr/bin/env python3
from pyarrow.fs import FileSystem, FileInfo
import sys
import shutil

def copy_half_file(uri1: str, uri2: str) -> None:
    fs1, path = FileSystem.from_uri(uri1)
    fs2, path2 = FileSystem.from_uri(uri2)

    f_info: FileInfo = fs1.get_file_info(path)  # Obtener información del archivo en fs1

    with fs1.open_input_file(path) as instream, \
         fs2.open_output_stream(path2) as outstream:
        # Mover el puntero de instream a la mitad del archivo
        instream.seek(f_info.size // 2)
        # Copiar el resto del archivo en outstream
        shutil.copyfileobj(instream, outstream)

if __name__ == "__main__":
    # Si no se proporcionan dos argumentos, se muestra un mensaje de error
    if len(sys.argv) != 3:
        print("Uso: {} <uri1> <uri2>".format(sys.argv[0]))
        sys.exit(1)

    copy_half_file(sys.argv[1], sys.argv[2])
    
    """)
    
# Enseñamos el contenido del archivo con bash
!cat /home/luser/copy_half_file.py


#! /usr/bin/env python3
from pyarrow.fs import FileSystem, FileInfo
import sys
import shutil

def copy_half_file(uri1: str, uri2: str) -> None:
    fs1, path = FileSystem.from_uri(uri1)
    fs2, path2 = FileSystem.from_uri(uri2)

    f_info: FileInfo = fs1.get_file_info(path)  # Obtener información del archivo en fs1

    with fs1.open_input_file(path) as instream,          fs2.open_output_stream(path2) as outstream:
        # Mover el puntero de instream a la mitad del archivo
        instream.seek(f_info.size // 2)
        # Copiar el resto del archivo en outstream
        shutil.copyfileobj(instream, outstream)

if __name__ == "__main__":
    # Si no se proporcionan dos argumentos, se muestra un mensaje de error
    if len(sys.argv) != 3:
        print("Uso: {} <uri1> <uri2>".format(sys.argv[0]))
        sys.exit(1)

    copy_half_file(sys.argv[1], sys.argv[2])
    
    

Ejecutamos el script con los siguientes argumentos:
- `hdfs:///user/luser/test.txt`
- `hdfs:///user/luser/test_half.txt`

In [72]:
%%bash

python3 copy_half_file.py hdfs://namenode:9000/user/luser/test.txt hdfs://namenode:9000/user/luser/test_half.txt
python3 filesystem_cat.py hdfs://namenode:9000/user/luser/test_half.txt

2025-01-01 16:55:22,897 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2025-01-01 16:55:27,207 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


na prueba de una linea de un archivo de texto.

FIN.


# 3. Probar el comando hdfs dfsadmin

Ahora habría que desconectarse del servidor de jupyter y conectarse al servidor de jupyter de hdadmin. Los pasos serían los mismos pero con hdadmin.

Comprobamos que estamos en el directorio de trabajo `/opt/bd`.

In [2]:
!pwd

/opt/bd


In [11]:
%%bash

# Creamos un directorio en HDFS
hdfs dfs -mkdir -p /user/hdadmin/limited

# Comprobamos que se ha creado
hdfs dfs -ls /user/hdadmin

Found 1 items
drwxr-xr-x   - hdadmin supergroup          0 2025-01-01 16:59 /user/hdadmin/limited


Limitamos la cuota de archivos del directorio `/user/hdadmin/limited` a 4 archivos.

In [32]:
%%bash

# Limitamos la cuota de ficheros en el directorio
hdfs dfsadmin -setQuota 4 /user/hdadmin/limited

# Comprobamos que se ha limitado
hdfs dfs -count -q /user/hdadmin/limited

           4              -1            none             inf            1            4                424 /user/hdadmin/limited


Copiamos cinco archivos de prueba al directorio `/user/hdadmin/limited` para comprobar que no se pueden copiar más de 4 archivos.

In [40]:
%%bash

# Copiamos el fichero /user/luser/test.txt en el directorio /user/hdadmin/limited cinco veces
hdfs dfs -cp -f /user/luser/test.txt /user/hdadmin/limited/test1.txt
hdfs dfs -cp -f /user/luser/test.txt /user/hdadmin/limited/test2.txt
hdfs dfs -cp -f /user/luser/test.txt /user/hdadmin/limited/test3.txt
hdfs dfs -cp -f /user/luser/test.txt /user/hdadmin/limited/test4.txt
hdfs dfs -cp -f /user/luser/test.txt /user/hdadmin/limited/test5.txt

cp: The NameSpace quota (directories and files) of directory /user/hdadmin/limited is exceeded: quota=4 file count=5
cp: The NameSpace quota (directories and files) of directory /user/hdadmin/limited is exceeded: quota=4 file count=5


CalledProcessError: Command 'b'\n# Copiamos el fichero /user/luser/test.txt en el directorio /user/hdadmin/limited cinco veces\nhdfs dfs -cp -f /user/luser/test.txt /user/hdadmin/limited/test1.txt\nhdfs dfs -cp -f /user/luser/test.txt /user/hdadmin/limited/test2.txt\nhdfs dfs -cp -f /user/luser/test.txt /user/hdadmin/limited/test3.txt\nhdfs dfs -cp -f /user/luser/test.txt /user/hdadmin/limited/test4.txt\nhdfs dfs -cp -f /user/luser/test.txt /user/hdadmin/limited/test5.txt\n'' returned non-zero exit status 1.

In [39]:
!hdfs dfs -ls /user/hdadmin/limited

Found 3 items
-rw-r--r--   3 hdadmin supergroup        106 2025-01-01 17:32 /user/hdadmin/limited/test1.txt
-rw-r--r--   3 hdadmin supergroup        106 2025-01-01 17:32 /user/hdadmin/limited/test2.txt
-rw-r--r--   3 hdadmin supergroup        106 2025-01-01 17:32 /user/hdadmin/limited/test3.txt


**Comportanmiento:**

El comando hdfs dfsadmin `bash -setQuota 4 /user/hdadmin/limited` establece una cuota de 4 para el directorio `/user/hdadmin/limited`. Sin embargo, esta cuota no se refiere al número de archivos, sino al número de nodos. En HDFS, cada archivo y cada directorio consume un nodo. Por lo tanto, si se tiene una cuota de 4 nodos, se pueden tener combinaciones de archivos y subdirectorios que sumen hasta 4 nodos en total.

Parece ser que el propio directorio `/user/hdadmin/limited` ya consume un nodo, por lo que solo se pueden tener 3 archivos en ese directorio.

# 4. Probar el comando hdfs fsck

In [43]:
!hdfs fsck /

Connecting to namenode via http://namenode:9870/fsck?ugi=hdadmin&path=%2F
FSCK started by hdadmin (auth:SIMPLE) from /172.18.0.2 for path / at Wed Jan 01 17:39:08 CET 2025


Status: HEALTHY
 Number of data-nodes:	4
 Number of racks:		3
 Total dirs:			42
 Total symlinks:		0

Replicated Blocks:
 Total size:	840850759 B
 Total files:	45
 Total blocks (validated):	55 (avg. block size 15288195 B)
 Minimally replicated blocks:	55 (100.0 %)
 Over-replicated blocks:	0 (0.0 %)
 Under-replicated blocks:	0 (0.0 %)
 Mis-replicated blocks:		0 (0.0 %)
 Default replication factor:	3
 Average block replication:	3.0
 Missing blocks:		0
 Corrupt blocks:		0
 Missing replicas:		0 (0.0 %)
 Blocks queued for replication:	0

Erasure Coded Block Groups:
 Total size:	0 B
 Total files:	0
 Total block groups (validated):	0
 Minimally erasure-coded block groups:	0
 Over-erasure-coded block groups:	0
 Under-erasure-coded block groups:	0
 Unsatisfactory placement block groups:	0
 Average block group size:	0.0
 Miss

Apagamos manualmente dos datanodes (datanode2 y datanode3) para simular un fallo en el sistema. Una vez apagados, tras unos 10 minutos comprobamos que no estén activos con `hdfs dfsadmin -report`.

In [46]:
!hdfs dfsadmin -report

Configured Capacity: 2162202353664 (1.97 TB)
Present Capacity: 2025699453797 (1.84 TB)
DFS Remaining: 2024067268608 (1.84 TB)
DFS Used: 1632185189 (1.52 GB)
DFS Used%: 0.08%
Replicated Blocks:
	Under replicated blocks: 55
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 2
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (2):

Name: 172.18.0.3:9866 (datanode1.hadoop-cluster)
Hostname: datanode1
Rack: /rack2
Decommission Status : Normal
Configured Capacity: 1081101176832 (1006.85 GB)
DFS Used: 847691873 (808.42 MB)
Non DFS Used: 13227495327 (12.32 GB)
DFS Remaining: 1012033634304 (942.53 GB)
DFS Used%: 0.08%
DFS Remaining%:

Una vez vemos que no están activos, volvemos a realizar el comando `hdfs fsck /` para comprobar el estado del sistema.

In [48]:
!hdfs fsck / | grep -i "Under-replicated"

Connecting to namenode via http://namenode:9870/fsck?ugi=hdadmin&path=%2F
 Under-replicated blocks:	55 (100.0 %)


Vemos que aparecen 55 bloques under-replicated.

Probamos a hacer un `-get` del fichero `/user/luser/libros/random_words.txt.bz2` para ver si funciona correctamente.

In [52]:
%%bash

hdfs dfs -get -f /user/luser/libros/random_words.txt.bz2
ls

hadoop
hadoop-3.3.6
random_words.txt.bz2


Vemos que los archivos se han copiado correctamente. Esto es debido a que HDFS está diseñado para mantener la disponibilidad de los datos incluso en situaciones de fallo, siempre y cuando exista al menos una réplica de cada bloque.

Ahora, vamos a crear un nuevo contenedor que sea `datanode6`, como hicimos con `datanode5` en la práctica 1. Una vez tenemos el contenedor creado, vamos a ver con `hdfs dfsadmin -report` que el nuevo datanode está activo.

In [55]:
!hdfs dfsadmin -report

Configured Capacity: 3243303530496 (2.95 TB)
Present Capacity: 3034918886372 (2.76 TB)
DFS Remaining: 3032375943168 (2.76 TB)
DFS Used: 2542943204 (2.37 GB)
DFS Used%: 0.08%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (3):

Name: 172.18.0.3:9866 (datanode1.hadoop-cluster)
Hostname: datanode1
Rack: /rack2
Decommission Status : Normal
Configured Capacity: 1081101176832 (1006.85 GB)
DFS Used: 847736832 (808.46 MB)
Non DFS Used: 14469103616 (13.48 GB)
DFS Remaining: 1010791981056 (941.37 GB)
DFS Used%: 0.08%
DFS Remaining%: 

Ahora, vamos a ver que se han replicado todos los bloques correctamente y ya no hay ningún bloque under-replicated.

In [56]:
!hdfs fsck / | grep -i "Under-replicated"

Connecting to namenode via http://namenode:9870/fsck?ugi=hdadmin&path=%2F
 Under-replicated blocks:	0 (0.0 %)
